In [1]:
import ROOT

Welcome to JupyROOT 6.12/07


In [2]:
%jsmva on

In [3]:
ROOT.TMVA.Tools.Instance()

<ROOT.TMVA::Tools object at 0x8ebecf0>

In [4]:
file1 = ROOT.TFile("samples.root")
tree_s = file1.Get("signal")
tree_b = file1.Get("bg")
fout = ROOT.TFile("BDTtest.root","RECREATE")

In [5]:
factory = ROOT.TMVA.Factory("TMVAClassification", TargetFile=fout,
                       V=False, Color=True, DrawProgressBar=True, Transformations=["I", "D", "P", "G", "D"],
                       AnalysisType="Classification")

In [6]:
#discriminating variables
dataset = "top" #the dataset name
loader  = ROOT.TMVA.DataLoader(dataset)
loader.AddVariable("trimass","F")
loader.AddVariable("12mass","F")
loader.AddVariable("13mass","F")
loader.AddVariable("23mass","F")

In [7]:
# define signal and background trees
loader.AddSignalTree(tree_s)
loader.AddBackgroundTree(tree_b)
loader.fTreeS = tree_s
loader.fTreeB = tree_b

In [8]:
# define additional cuts 
sigCut = ROOT.TCut("")
bgCut = ROOT.TCut("")

loader.PrepareTrainingAndTestTree(SigCut=sigCut, BkgCut=bgCut,
                    nTrain_Signal=0, nTrain_Background=0, SplitMode="Random", NormMode="NumEvents", V=False)

In [9]:
loader.DrawInputVariable("trimass")

In [10]:
factory.BookMethod(loader, ROOT.TMVA.Types.kBDT, "BDT",
    H=False, V=False, NTrees=850, MinNodeSize="2.5%", MaxDepth=3, BoostType="AdaBoost", AdaBoostBeta=0.5,
                   UseBaggedBoost=True, BaggedSampleFraction=0.5, SeparationType="GiniIndex", nCuts=20 )

<ROOT.TMVA::MethodBDT object ("BDT") at 0xa2a79c0>

Factory,Booking method: BDT


In [11]:
# self-explaining
factory.TrainAllMethods()
factory.TestAllMethods()
factory.EvaluateAllMethods()

In [12]:
factory.DrawOutputDistribution(dataset, "BDT")

In [13]:
factory.DrawROCCurve(dataset)

In [15]:
fout.Close()